In [86]:
import gym
import os
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
from gym import wrappers
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.optimizers import SGD, RMSprop, Adam, Adamax

In [87]:
env = gym.make('LunarLander-v2')

/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


## Observation samples for scaler

In [88]:
observation_samples = []

for n in range(100):
    observation = env.reset()
    observation_samples.append(observation)
    done = False
    while not done:
        action = np.random.randint(0, env.action_space.n)
        observation, reward, done, _ = env.step(action)
        observation_samples.append(observation)
        
observation_samples = np.array(observation_samples)

# Create scaler and fit
sc = StandardScaler()
sc.fit(observation_samples)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [89]:
# Observation sample
#
# array([ 0.00379438,  1.4016738 ,  0.38431674, -0.41094953, -0.00438996,
#       -0.08705341,  0.        ,  0.        ], dtype=float32)

## Creat monitor

In [90]:
#env = wrappers.Monitor(env, 'monitor-folder', force=True) # Saves cubed episodes

## Build Neural Networks - One for each action

In [91]:
def build_neural_network():
    model = Sequential()
    
    model.add(Dense(128, kernel_initializer='uniform', input_shape=(8,)))
    model.add(Activation('relu'))
    model.add(Dense(256, kernel_initializer='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(1, kernel_initializer='uniform'))
    model.add(Activation('linear')) #linear output so we can have range of real-valued outputs

    model.compile(loss='mse', optimizer='adamax')

    return model

## Combined Model

In [196]:
class Model:
    
    def __init__(self, env, scaler):
        self.env = env
        self.scaler = scaler
        self.models = []
        for i in range(env.action_space.n):
            model = build_neural_network()
            self.models.append(model) 

    def predict(self, state):
        state = self.scaler.transform(np.atleast_2d(state))
        preds = []
        for m in self.models:
            preds.append(m.predict(np.array(state), verbose=0)[0])
        return np.array(preds)                 

    def _fit(self, state, action, future_discounted_reward):
        state = self.scaler.transform(np.atleast_2d(state))
        model = self.models[action] # Different model depending on action
        
        #print(type(future_discounted_reward))
        
        if type(future_discounted_reward) is np.ndarray:
            print(future_discounted_reward)
            #future_dsicounted_reward = future_discounted_reward[0][action]
            x = 1
            print(np.array(future_discounted_reward[0][action]).shape)
            print(np.array(state).shape)
            model.fit(np.array(state), 
                      np.array(future_discounted_reward[0][action]), 
                      epochs=1, verbose=0)
        else:
            #future_dsicounted_reward = future_discounted_reward[action]
            print(future_discounted_reward)
            model.fit(np.array(state), np.array([future_discounted_reward]), epochs=1, verbose=0)
        #future_discounted_reward = future_discounted_reward[action]
        #print("State: ", state)
        #print("Reward: ", [future_discounted_reward])
   

    def action(self, state, EPSILON):
        if np.random.random() < EPSILON:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.predict(state))

In [197]:
def remember(prev_state, action, reward, state, done):
    memory.append((prev_state, action, reward, state, done))

In [198]:
def replay(memory, BATCH_SIZE):
    mini_batch = random.sample(memory, 100) # MAX 2000 in memory
    
    for prev_state, action, reward, state, done in mini_batch:
        if not done:
            next_state_pred_reward = reward + GAMMA * np.max(model.predict(state)[0])
        else:
            next_state_pred_reward = reward
        
        
        # PREDICT = INPUT ONE STATE -> 4 REWARDS (ARGMAX FOR ACTION)
        # FIT = 
        
        '''
    next_state_pred_reward:  -99.82350345313549
    
    state_pred_reward [[-0.21892993]
     [-0.12000708]
     [-0.14074107]
     [-0.5067307 ]]
    (1, 4)

    State:  [[-0.5119985  -2.050301    0.45516843 -0.79553884 -2.557886   -8.063909
      -0.12273416 -0.16050309]]
              
        '''
        
        #print("next_state_pred_reward: ", next_state_pred_reward)
        state_pred_reward = model.predict(state)
        #print("state_pred_reward", state_pred_reward)
        state_pred_reward = state_pred_reward.reshape(1, -1)
        state_pred_reward[0][action] = next_state_pred_reward
        #print("state_pred_reward_after", state_pred_reward)
        print(state_pred_reward.shape)
        model._fit(prev_state, action, state_pred_reward)

## Helpers

In [199]:
def plot(episode_rewards):
    running_avg = np.empty(len(episode_rewards))
    running_avg = list(map(lambda t: episode_rewards[max(0, t-25):(t+1)].mean(), range(len(episode_rewards))))
    plt.plot(running_avg)
    plt.title("Running Average")
    plt.show()

In [200]:
def mission_accomplished(episode_rewards, e):
    return episode_rewards[max(0, e-100):(e+1)].mean() >= 200

## Training

In [201]:
EPISODES = 10000
GAMMA = 0.99 # DISCOUNT
EPSILON = 1.0 / np.sqrt(1) # EXPLORATION RATE
EPSILON_DECAY_RATE = 0.001
EPSILON_MIN = 0.001
BATCH_SIZE = 10
episode_rewards = np.empty(1000)
from collections import deque
memory = deque(maxlen=2000)

In [202]:
model = Model(env, sc)

for e in range(EPISODES):
    
    EPSILON = 1.0 / np.sqrt(e+1)
    
    state = env.reset()
    done = False
    episode_reward = 0
    frames = 0
    
    while not done:
        
        # Perform action
        
        #env.render()
        action = model.action(state, EPSILON)
        prev_state = state
        state, reward, done, info = env.step(action)

        # Predict future discounted reward from current state and train model to find this mapping

        pred_reward = model.predict(state)
        future_discounted_reward = reward + GAMMA * np.max(pred_reward)
        model._fit(prev_state, action, future_discounted_reward)
        episode_reward += reward
        frames += 1
        
        remember(prev_state, action, future_discounted_reward, state, done)
        
    episode_rewards[e] = episode_reward
    
    print(len(memory))
    if len(memory) > 250: # 250
        print("Replay!")
        replay(memory, BATCH_SIZE)
    
    if len(memory) >= 1000:
        print("Clear!")
        memory.clear()
    
    if e % 5 == 0:
        print("Episode: ", e, "Itr", frames, "Reward:", episode_reward, "Epsilon: %.3f" % EPSILON, "Avg reward (25):", episode_rewards[max(0, e-25):(e+1)].mean())
    if mission_accomplished(episode_rewards, e):
        break

print("Avg reward for last 100 episodes:", episode_rewards[-100:].mean())
print("Total steps:", episode_rewards.sum())

plt.plot(episode_rewards)
plt.title("Rewards")
plt.show()

-0.43089758648579846
-1.5397417991988847
-1.5203512323190704
-2.086634413791113
-0.01425379091426521
0.11935966870356196
0.1910580358810239
-3.350988973836845
-1.8541350128452314
-0.794379190286927
0.3299435696975729
-0.2068191260982985
0.307316468854666
-0.5700394975650273
-1.8023034455257494
0.38142312991551874
-0.6668304083550936
0.7047614018623926
-0.5025364729628597
-0.8909181848943206
-0.3993445919996111
-2.0324370730272108
0.567244615809459
-0.6613359235366784
-1.3405324104571241
0.16843882574782357
-3.32380933145189
-0.416173420168044
0.43150459150367393
-1.2551293869086089
-0.02101671128161569
-0.548908348278594
-1.5238657540763143
0.6608510327839338
0.5551536895986577
-1.6561968396648388
0.7468405481494779
-1.3135159685497069
-0.35936598752088683
-1.8242678069372789
0.6371457786936946
0.7387851219654475
-0.2808076982317516
-1.6949708807974162
-0.542401483308779
-1.6639387532626346
-1.7541417957407486
-0.03748260248418887
-1.8553271131694693
-1.8574914167494103
0.4734112388862

ValueError: Error when checking target: expected activation_345 to have 2 dimensions, but got array with shape ()

In [ ]:
plot(episode_rewards)

In [ ]:
env.close()